# Training an image classifier

Let's put *FluxTraining.jl* to train a model on the MNIST dataset.

MNIST is simple enough that we can focus on the part where *FluxTraining.jl* comes in, the training. If you want to see examples of using FluxTraining.jl on larger datasets, see the documentation of [FastAI.jl](https://github.com/FluxML/FastAI.jl).

## Setup

*If you want to run this tutorial yourself, you can find the notebook file [here](https://github.com/lorenzoh/FluxTraining.jl/blob/master/docs/tutorials/mnist.ipynb)*.

To make data loading and batching a bit easier, we'll install some additional dependencies:

```julia
using Pkg; Pkg.add(["MLDataPattern", "DataLoaders"])
```

Now we can import everything we'll need.

In [1]:
using DataLoaders: DataLoader
using MLDataPattern: splitobs
using Flux
using FluxTraining

## Overview

There are 4 pieces that you always need to construct and train a [`Learner`](#):

- a model
- data
- an optimizer; and
- a loss function

## Building a `Learner`

Let's look at the **data** first.

*FluxTraining.jl* is agnostic of the data source. The only requirements are:

- it is iterable and each iteration returns a tuple `(xs, ys)`
- the model can take in `xs`, i.e. `model(xs)` works; and
- the loss function can take model outputs and `ys`, i.e. `lossfn(model(xs), ys)` returns a scalar


Glossing over the details as it's not the focus of this tutorial, here's the code for getting a data iterator of the MNIST dataset. We use `DataLoaders.DataLoader` to create an iterator of batches from our dataset.

In [2]:
xs, ys = (
    # convert each image into h*w*1 array of floats 
    [Float32.(reshape(img, 28, 28, 1)) for img in Flux.Data.MNIST.images()],
    # one-hot encode the labels
    [Float32.(Flux.onehot(y, 0:9)) for y in Flux.Data.MNIST.labels()],
)

# split into training and validation sets
traindata, valdata = splitobs((xs, ys))

# create iterators
trainiter, valiter = DataLoader(traindata, 128, buffered=false), DataLoader(valdata, 256, buffered=false);

Next, let's create a simple *Flux.jl* **model** that we'll train to classify the MNIST digits.

In [3]:
model = Chain(
    Conv((3, 3), 1 => 16, relu, pad = 1, stride = 2),
    Conv((3, 3), 16 => 32, relu, pad = 1),
    GlobalMeanPool(),
    Flux.flatten,
    Dense(32, 10),
)

Chain(Conv((3, 3), 1=>16, relu), Conv((3, 3), 16=>32, relu), GlobalMeanPool(), flatten, Dense(32, 10))

We'll use *categorical cross entropy* as a **loss function** and *ADAM* as an **optimizer**.

In [4]:
lossfn = Flux.Losses.logitcrossentropy
optim = Flux.ADAM();

Now we're ready to create a [`Learner`](#). At this point you can also add any callbacks, like [`ToGPU`](#) to run the training on your GPU if you have one available. Some callbacks are also [included by default](../callbacks/reference.md).

Since we're classifying digits, we also use the [`Metrics`](#) callback to track the accuracy of the model's predictions:

In [7]:
learner = Learner(model, (trainiter, valiter), optim, lossfn, Metrics(accuracy))

Learner()

## Training

With a `Learner` inplace, training is as simple as calling [`fit!`](#)`(learner, nepochs)`.

In [8]:
FluxTraining.fit!(learner, 10)

Epoch 1 TrainingPhase(): 100%|██████████████████████████| Time: 0:00:46


┌───────────────┬───────┬─────────┬──────────┐
│         Phase │ Epoch │    Loss │ Accuracy │
├───────────────┼───────┼─────────┼──────────┤
│ TrainingPhase │   1.0 │ 2.04939 │  0.25204 │
└───────────────┴───────┴─────────┴──────────┘


Epoch 1 ValidationPhase(): 100%|████████████████████████| Time: 0:00:02


┌─────────────────┬───────┬─────────┬──────────┐
│           Phase │ Epoch │    Loss │ Accuracy │
├─────────────────┼───────┼─────────┼──────────┤
│ ValidationPhase │   1.0 │ 1.70353 │   0.3821 │
└─────────────────┴───────┴─────────┴──────────┘


Epoch 2 TrainingPhase(): 100%|██████████████████████████| Time: 0:00:19


...

Epoch 10 TrainingPhase(): 100%|█████████████████████████| Time: 0:00:18


┌───────────────┬───────┬─────────┬──────────┐
│         Phase │ Epoch │    Loss │ Accuracy │
├───────────────┼───────┼─────────┼──────────┤
│ TrainingPhase │  10.0 │ 0.71684 │  0.79175 │
└───────────────┴───────┴─────────┴──────────┘


Epoch 10 ValidationPhase(): 100%|███████████████████████| Time: 0:00:02


┌─────────────────┬───────┬─────────┬──────────┐
│           Phase │ Epoch │    Loss │ Accuracy │
├─────────────────┼───────┼─────────┼──────────┤
│ ValidationPhase │  10.0 │ 0.68353 │  0.79449 │
└─────────────────┴───────┴─────────┴──────────┘


Learner()